**Изучение того как в python работать с интерфесми (как концепцией)**

# Иточники

1. https://realpython.com/python-interface/

Интерфейс - некоторая абстакция, которая позволяет определить какими методы должны быть обязательно реализованы в наследниках.

# Не формальное определение интефейса

Рассмотрим пример - класс парсер. При том там надо реализовать парсеры для pdf документов и для документов из электронной почнты (eml)

In [7]:
class InformalParserInterface:
    '''
        Класс который определят все общее 
        принадлежащее парсерам
    '''
    def load_data_source(self, path: str, file_name : str) -> str:
        '''
            Пусть пасерам надо доставать данные из различных
            файлов по пути path
        '''
        pass
    
    def extrac_text(self, full_file_name: str):
        '''
            Пусть всем парсерам надо уметь доставать данные из
            диррекстивно указанного пути
        ''' 
        pass

Итак, были описаны возможности которыми должен обладать любой парсер. Далее конктеретизация.

In [12]:
class PdfParser(InformalParserInterface):
    '''
        Конкредная реализация парсера для pdf документов
    '''
    def load_data_source(self, path: str, file_name:str) -> str:
        '''
            Определяем как именно должен работать парсинг
            для pdf документов
        '''
        pass
    
    def extract_text(self, full_file_path:str) -> dict:
        '''
            Тут также определяем как именно должен работать 
            парсинг для pdf документов
        '''
        pass
    
class EmlParser(InformalParserInterface):
    '''
        Конкретная реализация для документов электронной почты
    '''
    def load_data_source(self, path:str, file_name:str)->str:
        '''
            Определяем как именно должен работать парсинг для email документов
        '''
        pass
    
    def extract_text_from_email(self, full_file_path:str)->str:
        '''
            Метод определенный только для документов email,
            но он по прежнему определяет как должен работать
            парсинг для email документов
        '''
        pass

Убедимся, что подклассы созданные реализации являются классами-наследниками для базового класса `InformalParserInterface` с использованием функции `issubclass`

In [13]:
issubclass(PdfParser, InformalParserInterface)

True

In [14]:
issubclass(EmlParser, InformalParserInterface)

True

Идея, которую доносят в источнике 1, состоит в том, что хорошо чтобы `issubclass(EmlParser, InformalParserInterface)` возвращало `False` так как, мы не переопределили `extract_text` и `EmlParser` не может считаться полноценной реализацией интерфейса `InformalParserInterface`.

# Использование Метаклассов

Идея создания интерфейса через метакласс, сосотои в том, что интерфейс имеет метакласс, в котором переопределены. `__instancecheck__` и `__subclasscheck__` (подробнее об этих базовых методах можно узать тут https://github.com/Dranikf/knowledge_bank/blob/main/python_class_spesials/python_class_specials.ipynb в разделе "Методы->Методы метаклассов"). Приведенным ниже образом.

In [11]:
class ParserMeta(type):
    '''
        Мета-парсер который будет использоваться для
        создания парсеров
    '''
    def __subclasscheck__(cls, subclass):
        '''
            Все классы наследующие этот класс 
            в качесве мета класса будут, будут своими 
            экземплярами (в смысле функции issubclass) 
            воспринимать лишь те классы, в которых объявлены 
            и определены методы load_data_source и extract_text.
        '''
        return (
            hasattr(subclass, 'load_data_source') and
            callable(subclass.load_data_source) and
            hasattr(subclass, 'extract_text') and
            callable(subclass.extract_text)
        )
    
    
    def __instancecheck__(cls, instance):
        '''
            Все классы наследующие это класс в качестве
            мета класса, своими экземплярами будут воспринимпть
            лишь те объекты, классы которых воспинимаются
            наследниками
        '''
        return cls.__subclasscheck__(type(instance))
    

    
class UpdatedInformalParserInterface(metaclass = ParserMeta):
    '''
        Объявляем обновленный парсер-интерфейс
    '''
    pass

Рассмотрим, тот-же пример.

In [8]:
class PdfParserNew:
    '''
        Новая конкредная реализация парсера для pdf документов
    '''
    def load_data_source(self, path: str, file_name:str) -> str:
        '''
            Определяем как именно должен работать парсинг
            для pdf документов
        '''
        pass
    
    def extract_text(self, full_file_path:str) -> dict:
        '''
            Тут также определяем как именно должен работать 
            парсинг для pdf документов
        '''
        pass
    
class EmlParserNew:
    '''
        Конкретная реализация для документов электронной почты
    '''
    def load_data_source(self, path:str, file_name:str)->str:
        '''
            Определяем как именно должен работать парсинг для email документов
        '''
        pass
    
    def extract_text_from_email(self, full_file_path:str)->str:
        '''
            Метод определенный только для документов email,
            но он по прежнему определяет как должен работать
            парсинг для email документов
        '''
        pass

Проверяем результат выполнения функции `issubclass` для новосозданных классов.

In [12]:
issubclass(PdfParserNew, UpdatedInformalParserInterface)

hello


True

In [13]:
issubclass(EmlParserNew, UpdatedInformalParserInterface)

hello


False

Так формально `UpdatedInformalParserInterface` не является реализацией интерфейса `EmlParserNew`.

**Но такая реализация по прежнему не являтся правильной**<br>
Рассмотрим результат метода `__mro__` для `PdfParserNew` (`__mro__` - одно из <a href="https://github.com/Dranikf/knowledge_bank/blob/main/python_class_spesials/python_class_specials.ipynb">специальных полей</a>).

In [18]:
PdfParserNew.__mro__

(__main__.PdfParserNew, object)

Так в `__mro__` для класса `PdfParserNew`, не видно, что он как-либо связан с `UpdatedInformalParserInterface`. Такую ситуалию еще описывают, что `UpdatedInformalParserInterface` является виртуальным базовым классом для класса `PdfParserNew`.